In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [19]:
from src.inference import get_feature_store

In [20]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1) ),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-04 18:31:20,017 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 18:31:20,046 INFO: Initializing external client
2025-03-04 18:31:20,046 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 18:31:20,814 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215687
Fetching data from 2025-02-03 23:31:20.017514+00:00 to 2025-03-04 22:31:20.017514+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (21.85s) 


In [21]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-04 00:00:00+00:00,2,0
1,2025-02-04 01:00:00+00:00,2,0
2,2025-02-04 02:00:00+00:00,2,0
3,2025-02-04 03:00:00+00:00,2,0
4,2025-02-04 04:00:00+00:00,2,0
...,...,...,...
173938,2025-03-04 16:00:00+00:00,263,113
173939,2025-03-04 17:00:00+00:00,263,117
173940,2025-03-04 18:00:00+00:00,263,132
173941,2025-03-04 19:00:00+00:00,263,99


In [22]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 173943 entries, 0 to 174695
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         173943 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  173943 non-null  int32                  
 2   rides               173943 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [23]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [24]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 173943 entries, 0 to 174695
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         173943 non-null  datetime64[us]
 1   pickup_location_id  173943 non-null  int32         
 2   rides               173943 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [25]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [26]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-04 18:31:56,629 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 18:31:56,639 INFO: Initializing external client
2025-03-04 18:31:56,641 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 18:31:57,344 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215687
Fetching data from 2025-02-03 23:31:56.627158+00:00 to 2025-03-04 22:31:56.627158+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (6.01s) 


In [27]:
current_date

Timestamp('2025-03-04 23:31:56.627158+0000', tz='Etc/UTC')

In [28]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-04
1,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,3,2025-03-04
2,3,6,2,0,1,1,2,1,11,7,...,0,1,1,0,0,1,1,1,4,2025-03-04
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-03-04
4,0,0,1,0,0,1,2,2,1,2,...,3,4,1,5,2,1,1,0,7,2025-03-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,259,2025-03-04
247,0,0,0,0,0,0,1,4,2,0,...,1,6,1,1,0,0,0,1,260,2025-03-04
248,1,3,0,0,2,2,0,4,5,16,...,33,43,45,24,22,21,8,10,261,2025-03-04
249,1,3,1,1,3,19,63,126,177,156,...,87,82,103,65,37,16,14,9,262,2025-03-04


In [29]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-04 18:32:16,428 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 18:32:16,438 INFO: Initializing external client
2025-03-04 18:32:16,438 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 18:32:17,089 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215687


In [30]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [31]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,3.0
3,6,0.0
4,7,0.0
...,...,...
246,259,0.0
247,260,0.0
248,261,3.0
249,262,4.0


In [32]:
predictions.sort_values("predicted_demand", ascending=False).head(10)

,pickup_location_id,predicted_demand
122,132,251.0
128,138,121.0
132,142,60.0
217,230,59.0
44,48,46.0
104,114,43.0
174,186,43.0
64,68,41.0
236,249,38.0
75,79,35.0
